In [1]:
import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time
import pickle,h5py,json

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap,TSNE
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from scipy import fftpack
from scipy.signal import resample,correlate
from xgboost import XGBClassifier,XGBRegressor

import gym

import pandas_datareader as pdr
import urllib
from bs4 import BeautifulSoup
import networkx as nx
from lmfit import Model

import iexfinance as iex
from iexfinance.stocks import get_historical_data

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def play_episode(env,N_max=500,verbose=False):

    state = env.reset()

    S = []
    A = []
    R = []
    
    timesteps = 0
    for _ in range(N_max):

        if verbose: env.render()
        timesteps += 1
        action = env.action_space.sample()
        state_next,reward,done,info = env.step(action)

        S.append(state)
        A.append(action)
        R.append(reward)

        if done: break
        state = state_next

    return S,A,R,timesteps

In [3]:
env = gym.make('MountainCarContinuous-v0')
S,A,R,n = play_episode(env,verbose=False)
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
tf.reset_default_graph()

x_0 = tf.placeholder(dtype=tf.float32,shape=[None,2],name='input0')

x = tf.layers.Dense(units=6,activation=tf.nn.relu)(x_0)
x = tf.layers.Dense(units=5,activation=tf.nn.relu)(x)
y_rec = tf.layers.Dense(units=10,activation=tf.nn.softmax)(x)

log_y = tf.log(y_rec)

optimizer = tf.train.AdamOptimizer(0.1)
update = tf.gradients(log_y,tf.trainable_variables())

update_and_vars = list(zip(update, tf.trainable_variables()))
train_op = optimizer.apply_gradients(update_and_vars)

In [8]:
sess = tf.Session()

[array([0.09762701], dtype=float32),
 array([0.43037874], dtype=float32),
 array([0.20552675], dtype=float32),
 array([0.08976637], dtype=float32),
 array([-0.1526904], dtype=float32),
 array([0.29178822], dtype=float32),
 array([-0.12482557], dtype=float32),
 array([0.78354603], dtype=float32),
 array([0.92732555], dtype=float32),
 array([-0.23311697], dtype=float32),
 array([0.5834501], dtype=float32),
 array([0.05778984], dtype=float32),
 array([0.13608912], dtype=float32),
 array([0.85119325], dtype=float32),
 array([-0.85792786], dtype=float32),
 array([-0.8257414], dtype=float32),
 array([-0.9595632], dtype=float32),
 array([0.6652397], dtype=float32),
 array([0.5563135], dtype=float32),
 array([0.74002427], dtype=float32),
 array([0.9572367], dtype=float32),
 array([0.59831715], dtype=float32),
 array([-0.07704128], dtype=float32),
 array([0.56105834], dtype=float32),
 array([-0.76345116], dtype=float32),
 array([0.27984205], dtype=float32),
 array([-0.71329343], dtype=float32),